In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  
import os

from sklearn import preprocessing
import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import train_test_split

pd.set_option('display.max_rows', 500)

## Summary for next time I look at the data
1. need 41697600 data
2. how do I align all data: key should be 'row_id'

In [3]:
%%time
train = pd.read_csv('test_joined.csv', parse_dates=['timestamp'])

Wall time: 1min 10s


41697600 

## Take out cloud_coverage, precip_depth_1_hr, wind_direction, and  sea_level_pressure for now

In [4]:
train = train.drop(['year_built', 'floor_count', 'Unnamed: 0', 
                    'precip_depth_1_hr', 'cloud_coverage', 'sea_level_pressure', 'wind_direction'], axis=1)
# train = train.dropna()
train.isnull().sum()/train.shape[0]

row_id             0.000000
building_id        0.000000
meter              0.000000
timestamp          0.000000
site_id            0.000000
primary_use        0.000000
square_feet        0.000000
air_temperature    0.000551
dew_temperature    0.001488
wind_speed         0.002483
dtype: float64

## Prepocess
1. Label primary_use 
2. move y to the first column
3. Remove first 5 months in site 0
4. Add month, week day, day, hour

In [6]:
le = preprocessing.LabelEncoder()
train['primary_use'] = le.fit_transform(train['primary_use'])

move = ['timestamp'] + [x for x in train.columns if x not in ['meter_reading', 'timestamp']]
train = train[move]

In [7]:
# mask =(train['site_id']==0) & (train['timestamp'] < '2016-06-01')
# train = train[~mask]

In [7]:
%%time
## this may take mins
train['date_month'] = train['timestamp'].map(lambda x: x.month)
train['date_day'] = train['timestamp'].map(lambda x: x.day)
train['iso_week'] = train['timestamp'].map(lambda x: x.isocalendar()[1])
train['iso_weekend'] = train['timestamp'].map(lambda x: x.isocalendar()[2])
train['hour'] = train['timestamp'].map(lambda x: x.hour)

C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

Wall time: 6min 19s


C:\Users\spong\Anaconda3\envs\test\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [8]:
train.head()

,timestamp,row_id,building_id,meter,site_id,primary_use,square_feet,air_temperature,dew_temperature,wind_speed,date_month,date_day,iso_week,iso_weekend,hour
0,2017-01-01,0,0,0,0,0,7432,17.8,11.7,3.6,1,1,52,7,0
1,2017-01-01,1,1,0,0,0,2720,17.8,11.7,3.6,1,1,52,7,0
2,2017-01-01,2,2,0,0,0,5376,17.8,11.7,3.6,1,1,52,7,0
3,2017-01-01,3,3,0,0,0,23685,17.8,11.7,3.6,1,1,52,7,0
4,2017-01-01,4,4,0,0,0,116607,17.8,11.7,3.6,1,1,52,7,0


In [9]:
train.shape

(41498571, 15)

## Model on transformed y

In [9]:
%%time
X = train.iloc[:,1:]
# y = train.iloc[:,0].map(lambda x: np.log(x+1))
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=33)

Wall time: 27 s


In [26]:
building_list = train['building_id'].drop_duplicates()
building_list[-5:]

12050655    567
12967906    621
13361193    591
13918351    783
18997786    403
Name: building_id, dtype: int64

In [45]:
len(building_list)

1449

## Run models on test data

In [ ]:
prediction = pd.Series()

for sample in samples: 
    sample_df = train[train['building_id']==sample]
    X = sample_df.iloc[:,1:]
    
    bst = lgb.Booster(model_file = 'models/model{}.txt'.format(sample))
    
    prediction_on_X_train = bst.predict(X_train)
    prediction_on_X_train = pd.Series(prediction_on_X_train)
    prediction_on_X_train[prediction_on_X_train<0] = 0

    prediction_on_X_test = bst.predict(X_test)
    prediction_on_X_test = pd.Series(prediction_on_X_test)
    prediction_on_X_test[prediction_on_X_test<0] = 0
    
    y_train_list = pd.concat([y_train_list, y_train])
    y_test_list = pd.concat([y_test_list, y_test])

    prediction_train = pd.concat([prediction_train, prediction_on_X_train])
    prediction_test = pd.concat([prediction_test, prediction_on_X_test])

print('train: ', np.sqrt(mean_squared_log_error( y_train_list, prediction_train)))
print('test: ', np.sqrt(mean_squared_log_error( y_test_list, prediction_test)))